In [ ]:
import pandas as pd

rules_df = pd.read_csv("./inputs/ordering_rules.csv", sep="|", header=None)

with open("./inputs/updates.txt") as f:
    updates = f.readlines()

updates = [[int(u) for u in update.strip("\n").split(",")] for update in updates]

In [ ]:
def check_rule_for_update(rule: pd.Series, update: list):
    first = rule[0]
    second = rule[1]
    if first not in update or second not in update:
        return True

    first_index = update.index(first)
    second_index = update.index(second)

    if first_index < second_index:
        return True

    return False


def check_all_rules_for_update(rules_df: pd.DataFrame, update: list):
    for _, rule_se in rules_df.iterrows():
        if not check_rule_for_update(rule_se, update):
            return False
    return True


def get_list_middle_elements(lst):
    lst_len = len(lst)
    mid_index = lst_len // 2
    return lst[mid_index]

# Puzzle 1

In [ ]:
sum_page_number = 0
for update in updates:
    if check_all_rules_for_update(rules_df, update):
        sum_page_number += get_list_middle_elements(update)

In [ ]:
print(f"{sum_page_number=}")

# Puzzle 2

In [ ]:
from copy import deepcopy


class RuleNotForUpdate(Exception):
    pass


def get_rule_for_update(rule: pd.Series, update: list):
    first = rule[0]
    second = rule[1]
    if first in update and second in update:
        return first, second
    else:
        raise RuleNotForUpdate


def get_all_rules_for_update(rules_df: pd.DataFrame, update: list) -> list[tuple[int, int]]:
    rules = []
    for _, rule_se in rules_df.iterrows():
        try:
            rules.append(get_rule_for_update(rule_se, update))
        except RuleNotForUpdate:
            continue

    return rules


def get_first_page_of_first_rule(list_of_rules: list[tuple[int, int]]) -> int:
    for rule in list_of_rules:
        if not any([rule[0] == r[1] for r in list_of_rules]):
            return rule[0]


def get_pages_order_from_rules_list(list_of_rules: list[tuple[int, int]]) -> list[int, ...]:
    pages = []
    list_of_rules_copy = deepcopy(list_of_rules)
    if len(list_of_rules_copy) == 1:
        return list(list_of_rules_copy[0])
    first_page = get_first_page_of_first_rule(list_of_rules_copy)
    list_of_rules_copy = [rule_tuple for rule_tuple in list_of_rules_copy if rule_tuple[0] != first_page]
    pages.append(first_page)
    pages.extend(get_pages_order_from_rules_list(list_of_rules_copy))

    return pages

In [ ]:
incorect_updates = []

for update in updates:
    if not check_all_rules_for_update(rules_df, update):
        incorect_updates.append(update)

In [ ]:
sum_page_number = 0
for incorect_update in incorect_updates:
    all_rules_for_update = get_all_rules_for_update(rules_df, incorect_update)
    ordered_pages = get_pages_order_from_rules_list(all_rules_for_update)
    sum_page_number += get_list_middle_elements(ordered_pages)

In [ ]:
print(f"{sum_page_number=}")